In [42]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as rq
import json
import time

# Login to ResearchGate

In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.window import WindowTypes

driver = webdriver.Firefox()
driver.get("https://www.researchgate.net/login")

password = driver.find_element(By.ID, "input-password")
username = driver.find_element(By.ID, "input-login")

password.send_keys("kcaeDHT8uE5wYMe")
username.send_keys("matanki@my.hit.ac.il")
password.send_keys(Keys.RETURN)

In [44]:
fullname = []
gender = []
position = []
nation = []
institute = []
shanghai_ranking = []
depratment = []
disciplines = []
publications = []
citations = []
recommendations = []
top_cited_research = []
# top_cited_research_journal = []
interest_score = []
reads = []
h_index = []
URL = []

## List of institues & Shanghai ranking dict.

In [45]:
instLst = ["Holon-Institute-of-Technology", "Tel Aviv University", "Hebrew University of Jerusalem", "Bar Ilan University", "Technion-Israel Institute of Technology", "The-Open-University-of-Israel", "Ben-Gurion-University-of-the-Negev", "Ariel-University", "University-of-Haifa", "Weizmann-Institute-of-Science", "Academic College of Tel Aviv-Yafo", "Shamoon-College-of-Engineering"]

# dictionary
inst_rank = {
    "Holon-Institute-of-Technology":801,
    "Tel Aviv University":151,
    "Hebrew University of Jerusalem":77,
    "Bar Ilan University":301,
    "Technion-Israel Institute of Technology":83,
    "The-Open-University-of-Israel":201,
    "Ben-Gurion-University-of-the-Negev":401,
    "Ariel-University":801,
    "University-of-Haifa":501,
    "Weizmann-Institute-of-Science":83,
    "Academic College of Tel Aviv-Yafo":801,
    "Shamoon-College-of-Engineering":801
}

## Determine the gender by name using API

In [46]:
def genderFromName(string):
    name = string.split()
    name = name[0]
#     print(name) # taking only firstname
    url = "https://api.genderize.io?name="+name
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0'}
    file = rq.get(url, headers=headers)
    if str(file.content).find("female") == -1: #TODO: maybe a better way in tame consuming.
        return "male"
    else:
        return "female"

## Crawling

In [47]:
for x in instLst:
    driver.get("https://www.researchgate.net/institution/"+x+"/members")
    content = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[3]/div/div/nav/div[7]/a/span')
    index = bs(content.get_property("innerHTML"), "html.parser").get_text()
#     print(index) # Number of member pages for each institue
    
    limit = min(int(index) + 1, 101) # we don't mind more than 100*10 researchers per institue
    
    for i in range(1, limit): #int(index) + 1
        driver.get("https://www.researchgate.net/institution/"+x+"/members/"+str(i))
        p = driver.current_window_handle
        
        for j in range(1, 11):

            try:
                name = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[1]/div/div/h5/a")
                url = name.get_attribute('href')
                name = bs(name.get_property("innerHTML"), "html.parser").get_text()
                print(url)
                print(name)
            except:
                break #no more people to show
            
            print (genderFromName(name))
            print(x)
            print(inst_rank[x])
            try:
                department = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[2]/div/div[2]/ul/li/span")
                department = bs(department.get_property("innerHTML"), "html.parser").get_text()
                print(department)
            except:
                print("Nan")

            try: 
                disciplines = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[3]/div/div[2]")
                disciplines = bs(disciplines.get_property("innerHTML"), "html.parser").get_text("|", strip=True)
#                 A = [text for text in disciplines.stripped_strings] # getting list of disciplines instead of on string
                print(disciplines)
            except:
                print("Nan")
            
            # all data on the members' page was collected    
            # now we want to reach the web page of the sientist

            driver.switch_to.new_window(WindowTypes.TAB)
            driver.get(url+'/stats')
            
            try: 
                Research_Interest_Score = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[1]/div/div/div/div[1]/div")
                Research_Interest_Score = bs(Research_Interest_Score.get_property("innerHTML"), "html.parser").get_text()
                if Research_Interest_Score == '- -':
                    Research_Interest_Score = 0;
                print(Research_Interest_Score)
            except:
                print("Nan")

            try: 
                Reads = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[2]/div/div/div/div[1]/div")
                Reads = bs(Reads.get_property("innerHTML"), "html.parser").get_text()
                print(Reads)
            except:
                print("Nan")

            try: 
                Citations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[3]/div/div/div/div[1]/div")
                Citations = bs(Citations.get_property("innerHTML"), "html.parser").get_text()
                print(Citations)
            except:
                print("Nan")

            try: 
                Recommendations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[4]/div/div/div/div[1]/div")
                Recommendations = bs(Recommendations.get_property("innerHTML"), "html.parser").get_text()
                print(Recommendations)
            except:
                print("Nan")

            try: 
                hindex = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[1]/ul/li[1]/div/div/div/div[1]/div")
                hindex = bs(hindex.get_property("innerHTML"), "html.parser").get_text()
                print(hindex)
            except:
                print("Nan")

            try: 
                Top_cited_research = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[2]/div/div/div/div[2]/div/a/strong")
                Top_cited_research = bs(Top_cited_research.get_property("innerHTML"), "html.parser").get_text()
                print(Top_cited_research)
            except:
                print("Nan")

            try: 
                Position = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div[3]/ul/li[1]")
                Position = bs(Position.get_property("innerHTML"), "html.parser").get_text()
                word = Position.split()[0]
                if word == x.split()[0] or word == x.split('-')[0]:
                    Position = "Nan"
                print(Position)
            except:
                print("Nan")
            
            # now we've done scraping all relevent data on this sientinst
            # we close the tab and return to previous tab
            time.sleep(2)
            driver.close()
            driver.switch_to.window(p)
                
            print("\n")


#         time.sleep(2) # no need for delay, already taking some time..

https://www.researchgate.net/profile/Vladimir-Osipov-8
Vladimir Al. Osipov
male
Holon-Institute-of-Technology
801
Department of physics
Nonlinear Dynamics|Statistics|Probability
388.7
3,402
762
61
14
p-Adic Models of Ultrametric Diffusion Constrained by Hierarchical Energy Landscapes
PhD


https://www.researchgate.net/profile/Ronens-Sosnik
Ronens Sosnik
male
Holon-Institute-of-Technology
801
Faculty of Engineering
Brain Imaging|Cognitive Neuroscience|Neuroimaging
694.5
8,872
1,327
11
15
Transformation from temporal to rate coding in a somatosensory thalamocortical pathway
Senior Lecturer


https://www.researchgate.net/profile/Jacob-Gavan
Jacob. Gavan
male
Holon-Institute-of-Technology
801
Digital Signal Processing
Nan
99.0
2,775
351
0
10
Effects of desensitization on mobile radio system performance, part II: Quantitative analysis
Full Professor   Fellow IEEE


https://www.researchgate.net/profile/Y-Berkovich
Y. Berkovich
male
Holon-Institute-of-Technology
801
Faculty of Engineering
Pow

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:186:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.jsm:440:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.jsm:445:13
assert.open@chrome://remote/content/shared/webdriver/Assert.jsm:153:4
GeckoDriver.prototype.close@chrome://remote/content/marionette/driver.js:2103:10
despatch@chrome://remote/content/marionette/server.js:306:40
execute@chrome://remote/content/marionette/server.js:279:16
onPacket/<@chrome://remote/content/marionette/server.js:252:20
onPacket@chrome://remote/content/marionette/server.js:253:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.js:500:20


בעיה: אותם נתיבים class ים למאפיינים שונים

In [22]:
content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-xl nova-legacy-e-text--family-display nova-legacy-e-text--spacing-none nova-legacy-e-text--color-grey-900"]').text
fullname += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
degree += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-grey-700"]').text
nation += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
institute += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
department += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
disciplines += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"]').text
citations += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-xxxl nova-legacy-e-text--family-display nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"]').text
recommendations += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"]').text
h_index += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
top_cited_research += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
top_cited_research_journal += [content]

In [13]:
df = pd.DataFrame({"שם הספר":fullname, "מחבר":author, "עלות (בתוספת מע\"מ)":price})
df

,שם הספר,מחבר,"עלות (בתוספת מע""מ)"
0,נשף המשוגעות,ויקטוריה מס,79.2 ₪
1,מיומנה של אישה מתה,גואל פינטו,88.2 ₪
2,אהובת הצייר,דר פלוכט סימונה ואן,84.6 ₪
3,שיעורים בכימיה,בוני גרמוס,93.6 ₪
4,והגיענו לזמן הזה,עידן סגר,88.2 ₪
5,על השעות השבורות,ריקי כהן,68.4 ₪
6,צרוב בגוף,ישראל המאירי,75.6 ₪
7,אילו נולד איטלקי,שי אספריל,84.6 ₪
8,אדונית השקרנים,אן פאצט,88.2 ₪
9,העמק הירוק האחרון,מארק סאליבן,88.2 ₪


In [14]:
df.to_csv('book_prices')